<a href="https://colab.research.google.com/github/shukri0/workice-translations/blob/master/RVC_with_zip_loading_gg_AIHUB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title GPU Check
!nvidia-smi

In [ ]:
#@title Install Dependencies
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!apt -y install -qq aria2

In [ ]:
#@title Clone Github Release
%cd /content/
!wget https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/releases/download/20230428updated/updated20230428.zip
!mkdir /content/Retrieval-based-Voice-Conversion-WebUI
!unzip -d /content/Retrieval-based-Voice-Conversion-WebUI -b updated20230428.zip
!rm updated20230428.zip
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title Download the Base Model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Run RVC GUI
#@markdown After starting this cell, click on the public URL it generates. <br>
#@markdown The Index and Npy files can be found in `/content/Retrieval-based-Voice-Conversion-WebUI/logs/MODELNAME/`  For inference, right click them both and hit "Copy Path".

%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs.
!python3 infer-web.py --colab --pycmd python3

# Training

In [ ]:
#@title Load dataset Zip to /content/dataset

#@markdown Dataset Zip Location:
DATASET = "/content/drive/MyDrive/dataset/rihannamodel.zip"  #@param {type:"string"}

!mkdir -p /content/dataset
!unzip -d /content/dataset -B {DATASET}

In [ ]:
#@title Clean up Dataset filenames
!ls -a /content/dataset/
!rename 's/(\w+)\.(\w+)~(\d*)/$1_$3.$2/' /content/dataset/*.*~*

# Saving/Loading Models

In [ ]:
import os
import os.path
import shutil
#@title Load model Zip from Google Drive
#@markdown Insert the model name and steps below:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

#@markdown Model Zip Location

MODELLOCATION = "/content/drive/MyDrive/RVC/kanye.zip"  #@param {type:"string"}

#@markdown Model Name
MODELNAME = "kanye"  #@param {type:"string"}

#@markdown Loading for subsequent training? (Will load G and D files, which are huge but not used for inference)
GDLoading = False #@param {type:"boolean"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!mkdir -p /content/unzips/{MODELNAME}/
!unzip -d /content/unzips/{MODELNAME} -B {MODELLOCATION}

# Set your source and destination directories
source_dir = os.path.join("/content/unzips/", MODELNAME)


# List all files in the source directory
files = os.listdir(source_dir)

# Filter files starting with 'G_' and 'D_'
g_files = [f for f in files if ("G_") in f]
d_files = [f for f in files if ("D_") in f]
model_files = [f for f in files if f.startswith(MODELNAME) and has_numbers(f) and "G_" not in f and "D_" not in f]

# Find the highest numbered G_ and D_ files
# g_max = max(g_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
# d_max = max(d_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))

if model_files:
  model_max = max(model_files, key=lambda x: (x.split(MODELNAME)[-1].split(".")[0]))

# print(g_max)

# GNumber = (g_max.split("_")[2].split(".")[0])
# DNumber = (d_max.split("_")[2].split(".")[0])
if model_files:
  MODELSTEP = (model_max.split(MODELNAME)[1].split(".")[0])
else:
  MODELSTEP = "no"

#@markdown If this stops working for whatever reason please spam ping me as much as possible at discord.gg/aihub
print(MODELSTEP)


if GDLoading:
  g_files = [f for f in files if ("G_") in f]
  g_max = max(g_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
  GNumber = (g_max.split("_")[2].split(".")[0])
  !mv /content/unzips/{MODELNAME}/*D_{GNUMBER}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELSTEP}.pth
  !mv /content/unzips/{MODELNAME}/*G_{MODELSTEP}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELSTEP}.pth

!mv /content/unzips/{MODELNAME}/*.index /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/
!mv /content/unzips/{MODELNAME}/*.npy /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/
!mv /content/unzips/{MODELNAME}/{MODELNAME}*.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth
!mv /content/unzips/{MODELNAME}/{MODELNAME}{MODELSTEP}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth
!mv /content/unzips/{MODELNAME}/{MODELNAME}.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth
!mv /content/unzips/{MODELNAME}/{MODELNAME}/*.index /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/
!mv /content/unzips/{MODELNAME}/{MODELNAME}/*.npy /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/
!mv /content/unzips/{MODELNAME}/{MODELNAME}/{MODELNAME}*.pth /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth

!rm -r /content/unzips/{MODELNAME}/

In [ ]:
import os
import os.path
import shutil
#@title Save model Zip to Drive

#@markdown Insert the model name and steps below:

#@markdown Model Zip Save Location

MODELLOCATION = "/content/drive/MyDrive/RVC"  #@param {type:"string"}

#@markdown Model Name
MODELNAME = "rihanna"  #@param {type:"string"}
#@markdown Saving for subsequent training? (Will save G and D files, which are huge but not used for inference)
GDSaving = False #@param {type:"boolean"}



# Set your source and destination directories
source_dir = os.path.join("/content/Retrieval-based-Voice-Conversion-WebUI/logs/", MODELNAME)


# List all files in the source directory
files = os.listdir(source_dir)

# Filter files starting with 'G_' and 'D_'
g_files = [f for f in files if f.startswith("G_")]
d_files = [f for f in files if f.startswith("D_")]

# Find the highest numbered G_ and D_ files
g_max = max(g_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
d_max = max(d_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))

GNumber = (g_max.split("_")[1].split(".")[0])
DNumber = (d_max.split("_")[1].split(".")[0])

!mkdir -p /content/zips/{MODELNAME}/

if GDSaving:
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{DNumber}.pth /content/zips/{MODELNAME}/{MODELNAME}_G_{GNumber}.pth
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{DNumber}.pth /content/zips/{MODELNAME}/{MODELNAME}_D_{DNumber}.pth

!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/zips/{MODELNAME}/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/zips/{MODELNAME}/
!cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/zips/{MODELNAME}/{MODELNAME}{GNumber}.pth

%cd /content/zips/{MODELNAME}/
!mkdir -p {MODELLOCATION}/
!zip -r {MODELNAME}.zip *
!mv {MODELNAME}.zip {MODELLOCATION}/
!rm -r /content/zips/{MODELNAME}
%cd /content/Retrieval-based-Voice-Conversion-WebUI/


In [ ]:
import os
import os.path
import shutil
#@title Manual Zip loading from Google Drive

#@markdown Model Zip Location

MODELLOCATION = "/content/drive/MyDrive/RVC/kanye.zip"  #@param {type:"string"}

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!mkdir -p /content/unzips/{MODELNAME}/
!unzip -d /content/unzips/{MODELNAME} -B {MODELLOCATION}

# Set your source and destination directories
source_dir = os.path.join("/content/unzips/", MODELNAME)
#@markdown If you use this, you'll have to move the files manually.

# Other

In [ ]:
#@title Clone Github Repository

!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights
!git pull

In [ ]:
#@title Change Python Version
#@markdown Run this, and pick the number that corresponds to "Python 3.8". (idk if t his actually works yett so if it doesnt dont do it idk how to turn off autosaving on cocle colab)
!python3 --version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3 --help
!sudo apt install python3-pip
!python3 --version

In [ ]:
#@title 手动预处理（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 采样率
BITRATE = 48000  #@param {type:"integer"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}

!python3 trainset_preprocess_pipeline_print.py /content/dataset {BITRATE} {THREADCOUNT} logs/{MODELNAME} True


In [ ]:
#@title 手动提取特征（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的进程数
THREADCOUNT = 8  #@param {type:"integer"}
#@markdown 音高提取算法
ALGO = "harvest"  #@param {type:"string"}

!python3 extract_f0_print.py logs/{MODELNAME} {THREADCOUNT} {ALGO}

!python3 extract_feature_print.py cpu 1 0 0 logs/{MODELNAME}


In [ ]:
#@title 手动训练（不推荐）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 使用的GPU
USEGPU = "0"  #@param {type:"string"}
#@markdown 批大小
BATCHSIZE = 32  #@param {type:"integer"}
#@markdown 停止的epoch
MODELEPOCH = 3200  #@param {type:"integer"}
#@markdown 保存epoch间隔
EPOCHSAVE = 100  #@param {type:"integer"}
#@markdown 采样率
MODELSAMPLE = "48k"  #@param {type:"string"}
#@markdown 是否缓存训练集
CACHEDATA = 1  #@param {type:"integer"}
#@markdown 是否仅保存最新的ckpt文件
ONLYLATEST = 0  #@param {type:"integer"}

!python3 train_nsf_sim_cache_sid_load_pretrain.py -e lulu -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g {USEGPU} -te {MODELEPOCH} -se {EPOCHSAVE} -pg pretrained/f0G{MODELSAMPLE}.pth -pd pretrained/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA}


In [ ]:
#@title 删除其它pth，只留选中的（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*.pth

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}

In [ ]:
#@title 清除项目下所有文件，只留选中的模型（慎点，仔细看代码）
#@markdown 模型名
MODELNAME = "lulu"  #@param {type:"string"}
#@markdown 选中模型epoch
MODELEPOCH = 9600  #@param {type:"integer"}

!echo "备份选中的模型。。。"
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth /content/{MODELNAME}_D_{MODELEPOCH}.pth
!cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth /content/{MODELNAME}_G_{MODELEPOCH}.pth

!echo "正在删除。。。"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!rm -rf /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/*

!echo "恢复选中的模型。。。"
!mv /content/{MODELNAME}_D_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_{MODELEPOCH}.pth
!mv /content/{MODELNAME}_G_{MODELEPOCH}.pth /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_{MODELEPOCH}.pth

!echo "删除完成"
!ls /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}